In [6]:
# Libraries
from pathlib import Path
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os

In [7]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("–disable-extensions")

In [8]:
# Earthquake Column headers]
table_1_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10']
table_2_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12']
table_3_headers = ['col_1', 'col_2', 'col_3', 'col_4']
table_4_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12']
table_5_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5']
table_6_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
table_7_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10']
table_8_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6']
table_9_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7']
table_10_headers = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6']



# Main function
def wildfire_csv(html_file):
    # File and event details
    file_name = html_file.split('/')[-1].split('.')[0]
    event_id, episode, event_type = file_name.split('_')[0:3]
    print(event_id, episode, event_type)

    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(html_file)


    # Define table configurations (XPaths and column limits)
    table_configs = [
        ('//*[@id="tab_responsive"]/table/tbody', 8),
        ('//*[@id="ctl00_CPH_GridViewEpisodes"]/tbody', 10),
        ('//*[@id="graph_eq"]/table/tbody/tr/td/table/tbody', 0),
        ('//*[@id="ctl00_CPH_GridShakemaps"]/tbody', 10),
        ('//*[@id="provinces"]/table/tbody', 4),
        ('//*[@id="cities"]/table/tbody', 7),
        ('//*[@id="airports"]/table/tbody', 9),
        ('//*[@id="ports"]/table/tbody', 5),
        ('//*[@id="dams"]/table/tbody', 6),
        ('//*[@id="nuclear"]/table/tbody', 5)
    ]

    # Process each table
    tables_data = []
    for i, (xpath, col_limit) in enumerate(table_configs):
        table_data = extract_table_data(driver, event_id, episode, xpath, col_limit)
        tables_data.append(table_data)
    for row in tables_data:
        print(row)
    driver.quit()

    # Define CSV destination
    destination_folder = 'CSV'
    absolute_folder_path = Path(destination_folder).resolve()

    # Generate CSV files
    for i, table_data in enumerate(tables_data):
        csv_file_path = f'{absolute_folder_path}\{i}.csv'
        file_exists = os.path.exists(csv_file_path)
        header = eval(f"table_{i+1}_headers")
        print(i,header)
        print(table_data)
        df = pd.DataFrame(table_data,columns=header)
        df.head()
        df.to_csv(csv_file_path, mode='a' if file_exists else 'w', header=not file_exists, index=False)
        print(f"Data {'appended to' if file_exists else 'written to'} {csv_file_path}")

# Function to extract table data based on XPath and the number of columns required
def extract_table_data(driver, event_id, episode, xpath, col_limit):
    table_data = []
    if xpath == '//*[@id="alert_summary_left"]/table/tbody' or xpath == '//*[@id="tab_responsive"]/table/tbody' :
        try:
            temp_tbl=[]
            table = driver.find_element(By.XPATH, xpath)
            rows = table.find_elements(By.XPATH, './/tr')
            event_summary_text = driver.find_element(By.XPATH, '//*[@class="p_summary"][1]').text
            for row in rows:
                columns = row.find_elements(By.XPATH, './/td')
                row_data = [column.text for column in columns]
                if len(row_data)>=1:
                    temp_tbl.append(row_data)
            table_data = [[event_id, episode,*[temp_tbl[i][1] for i in range(1,col_limit)]
                        ,event_summary_text]]
        except NoSuchElementException:
            print(f"Event summary not founds or table with XPath '{xpath}' not found.")
        return table_data
    else:
        try:
            table = driver.find_element(By.XPATH, xpath)
            rows = table.find_elements(By.XPATH, './/tr')
            for row in rows:
                columns = row.find_elements(By.XPATH, './/td' if col_limit else './/th | .//td')
                row_data = []
                for col in columns:
                    cell_text = col.text.strip()

                    # If the cell is empty, look for an image
                    if not cell_text:
                        img_element = col.find_element(By.TAG_NAME, 'img') if col.find_elements(By.TAG_NAME, 'img') else None
                        if img_element:
                            # Try to get the title attribute
                            image_title = img_element.get_attribute('title')
                            if image_title:
                                row_data.append(image_title)  # Add the image title if present
                            else:
                                # If no title, extract from the image URL
                                image_url = img_element.get_attribute('src')
                                image_code = image_url.split('/')[-1].split('.')[0]  # Extract "8p" from URL
                                row_data.append(image_code)  # Add the extracted part from the URL
                        else:
                            row_data.append("")  # Leave it blank if no image is found
                    else:
                        # If the cell contains text, just add it
                        row_data.append(cell_text)

                if len(row_data) >= 1:
                    table_data.append([event_id, episode] + row_data[:col_limit if col_limit else len(row_data)])
        except NoSuchElementException:
            print(f"Table with XPath '{xpath}' not found.")
    return table_data

In [9]:
def list_filenames(folder_path):
    try:
        # Get a list of all files in the specified folder
        filenames = os.listdir(folder_path)
        
        return filenames
    
    except FileNotFoundError:
        print(f"Folder not found: {folder_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

def main():
    folder_path = "latest_htmls/Earthquakes"
    absolute_folder_path = Path(folder_path).resolve()
    filenames = list_filenames(absolute_folder_path)

    for file in filenames:
        absolute_file_path = absolute_folder_path / file
        wildfire_csv(f"file:///{absolute_file_path.as_posix()}")


In [10]:
if __name__ == "__main__":
    main()

1446591 1594407 Earthquakes
Table with XPath '//*[@id="ctl00_CPH_GridShakemaps"]/tbody' not found.
Table with XPath '//*[@id="nuclear"]/table/tbody' not found.
[['1446591', '1594407', '28.6 Km', '24.2283 , 121.6792', '21 Sep 2024 10:51 UTC\n21 Sep 2024 18:51 Local', '440 thousand in MMI IV', 'NEIC us6000ntgm', '21 Sep 2024 11:11 UTC', '3.6 (Taiwan)', 'This earthquake is expected to have a low humanitarian impact based on the magnitude and the affected population and their vulnerability.']]
[['1446591', '1594407', '1594407', 'EQ', '0', '21 Sep 2024 10:51', '4.8M, 28.6km', '4.14', '440 thousand (in 100km)', '', '00:20', 'NEIC']]
[['1446591', '1594407', 'Radius', 'Population'], ['1446591', '1594407', '100 km', '11.8 million people'], ['1446591', '1594407', '75 km', '710000 people'], ['1446591', '1594407', '50 km', '360000 people'], ['1446591', '1594407', '20 km', '15000 people'], ['1446591', '1594407', '10 km', '1300 people'], ['1446591', '1594407', '5 km', '<1000 people'], ['1446591', '1